# Descriptor

### 학습 내용
- descriptor가 무엇인지 이해하고 어떻게 효율적으로 동작하는지 이해한다.
- data descriptor vs non-data descriptor 의 개념적 차이와 세부구현의 차이를 분석
- descriptor를 활용한 코드 재사용 방법
- 좋은 활용 예

학습에 앞서 먼저 descriptor의 syntax를 알아보고자 한다.
https://docs.python.org/3/howto/descriptor.html<br>
위 사이트에서 공부하였음

가장 쉬운 예제는 아래와 같다<br>
A descripotr that returns a constant

In [1]:
class Ten:
    def __get__(self, obj, objtype=None):
        return 10

To use the desciptor, it must be stored as a class Variable in another class<br>

In [2]:
class A:
    x = 5
    y = Ten()

In [3]:
a = A()
print(a.x)
print(a.y)

5
10


In the a.x attribute lookup, the dot operator finds the key x<br>
and the value 5 in the class dictionary.<br>

In the a.y lookup, the dot operator finds a descripotr instance, <br>
recognized by its \_\_get__ method, and calls that method which returns 10<br>

Note that the value 10 is not stored in either the class or the instance dic<br>
Instead, the value 10 in computed on demand

and we're gonna see dynamic lookup

Interesting descriptor typically run computation instead of returning constant<br>

In [5]:
import os

class DirectorySize:
    def __get__(self, obj, objtype=None):
        return len(os.listdir(obj.dirname))

class Directory:
    size = DirectorySize()

    def __init__(self, dirname):
        self.dirname = dirname

In [7]:
s = Directory('../')
j = Directory('../joono')

print(s.size)
print(j.size)

9
8


위와 같은 예제를 살펴보면 \_\_get__ method가 어떻게 동작하는지, 그 목적이 무엇인지 알 수 있다.<br>

descriptor가 가장 자주 사용되는 부분은 instance의 data에 접근을 제어하는 일이다.<br>
Descriptor는 public attribute에 저장되고 실제 data는 다른 instance dict에 저장된다.<br>
그리고 descriptor의 \_\_get__, \_\_set__ methods는 public attribute에 접근할 때 호출된다.<br>

아래의 예제에서 _age_는 public attribute,<br>
_\_age_는 private attribute이다.<br>
그리고 age에 access할 때 마다 descriptor가 lookup, update를 log한다.

In [8]:
import logging

logging.basicConfig(level=logging.INFO)

class LoggedAgeAccess:

    def __get__(self, obj, objtype=None):
        value = obj._age
        logging.info('Accessing %r giving %r', 'age', value)
        return value

    def __set__(self, obj, value):
        logging.info('Updating %r to %r', 'age', value)
        obj._age = value

class Person:

    age = LoggedAgeAccess()             # Descriptor instance

    def __init__(self, name, age):
        self.name = name                # Regular instance attribute
        self.age = age                  # Calls __set__()

    def birthday(self):
        self.age += 1

아래 결과를 보면 regular attribute인 name에 access할 때는 아무런 log도 남지 않지만<br>
Descriptor로 정의된 age에 접근할 때면 desciptor가 log를 남긴다.

_obj_ parameter는 g나 s같은 instance of Dictionary이고 이는 target directory가 무엇인지<br>
\_\_get__ method 에게 알려주는 역할을 한다.<br>
_objtype_ parameter는 class Dictionary이다.<br>

In [9]:
mary = Person('Mary M', 30)
david = Person('David D', 40)

INFO:root:Updating 'age' to 30
INFO:root:Updating 'age' to 40


In [10]:
print(vars(mary))
print(vars(david))

{'name': 'Mary M', '_age': 30}
{'name': 'David D', '_age': 40}


In [11]:
mary.age

INFO:root:Accessing 'age' giving 30


30

In [12]:
david.age

INFO:root:Accessing 'age' giving 40


40

그러나 위와 같은 방법의 문제점은 딱 age 하나 밖에 descriptor를 설정하지 못한다는 단점이 있다.
아래에서 위와 같은 문제를 해결해보자

descriptor를 사용할 때 각 descriptor에게 어떤 변수명이 사용됐는지 알려줄 수 있다.<br>
_Person_ class는 2개의 name, age라는 descriptor instance를 가지고 있다.<br>

_Person_ class가 정의될 때, \_\_set_name__이라는 함수를 callback한다.<br>
name, age에게 이들의 public, private 변수명을 알려주기 위해서이다.

In [19]:
import logging

logging.basicConfig(level=logging.INFO)

class LoggedAccess:
    def __set_name__(self, owner, name):
        self.public_name = name
        self.private_name = '_' + name

    def __get__(self, instance, owner):
        value = getattr(instance, self.private_name)
        logging.info(f'Accessing {self.public_name} to {value}')
        return value

    def __set__(self, instance, value):
        logging.info(f'Updating {self.public_name} to {value}')
        setattr(instance, self.private_name, value)

class Person:
    name = LoggedAccess()
    age = LoggedAccess()

    def __init__(self, name, age):
        self.name = name
        self.age = age

    def birthday(self):
        self.age += 1

Person classr가 정의될 때, 각 public, private name들을 저장하기 위해 \_\_set_name__이 callback 되었다.<br>
vars 함수를 통해 정말 그렇게 되었는지 확인해보자.<br>

descriptor는 'dot operator'에 의해서만 invoke 된다.<br>
따라서 아래의 두 코드에서는 log가 발생되지 않는다.

In [15]:
vars(vars(Person)['name'])

{'public_name': 'name', 'private_name': '_name'}

In [17]:
vars(vars(Person)['age'])

{'public_name': 'age', 'private_name': '_age'}

그러나 아래의 코드에서는 모두 dot operator가 사용되므로 log가 발생했다.

In [ ]:
pete = Person('Pete P', 10)
kate = Person('Kate K', 20)

descriptor instance에는 private name만 contain하게 된다.

In [22]:
print(vars(pete))
print(vars(kate))

{'_name': 'Pete P', '_age': 10}
{'_name': 'Kate K', '_age': 20}


## Closing thought
- descriptor는 \_\_get__, \_\_set__, \_\_delete__ method를 정의하는 class를 지칭하며,<br>
optional 하게 \_\_set_name__ 도 포함할 수 있다.
- descriptor는 dot operator에 의해서만 invoke 된다.
- 오직 class variable로 선언될 때만 사용 가능하다. instance로 선언되면 아무런 효과도 없다.
- descriptor의 motivation은 attrubute를 lookup할 때 발생하는 상황에 대하여 통제할 수 있도록<br>
 class variable에 objects를 저장할 수 있는 hook을 제공하는 것이다.<br>
- classmethod, staticmethod, property등등이 descriptor로서 사용된다.